This is a type of location optimization analysis, specifically finding the optimal location of facilites on a network. These are two types of Set-Coverage analysis implemented in **Python**:

### The Maximal Covering Location Problem (MCLP)
#### Objective: The Maximal Covering Location Problem determines the location of P facilities in order to maximize the demand covered within a pre-specified maximum distance coverage.

more information on GOSTNets Optimization can be found in the wiki: https://github.com/worldbank/GOST_PublicGoods/wiki/GOSTnets-Optimization

In [4]:
import geopandas as gpd
import pandas as pd
import os, sys, time
sys.path.append(r'../../../GOSTNets/GOSTNets')
import GOSTnet_Optimization as gn
import importlib
import networkx as nx
import osmnx as ox

In [9]:
pth = r'../../../../lima_optimization_output'

### load OD matrix

In [21]:
OD_df = pd.read_csv(os.path.join(pth, 'saved_OD.csv'),index_col=0)

### read origins snapped csv

In [10]:
origins_snapped_df =  pd.read_csv(os.path.join(pth, 'origins_snapped.csv'))
origins_snapped_df[:3]

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,NN,NN_dist
0,0,1,3.0,0.003205,2.860800e-07,0,1078.0,-12.24839,-76.91749,POINT (-76.91749 -12.24839),6528,21.347245
1,1,2,3.0,0.008010,2.565700e-06,1,2374.0,-12.24319,-76.92767,POINT (-76.92766999999999 -12.24319),5270,37.506920
2,2,3,3.0,0.010808,2.141710e-06,2,367.0,-12.24444,-76.91685,POINT (-76.91685 -12.24444),1921,6.260416


In [14]:
#create a series of the unique Nearest Node (NN) and its corresponding population value
origins_w_demands_series = pd.Series(origins_snapped_df.Population.values,index=origins_snapped_df.NN)
len(origins_w_demands_series)

707

In [17]:
#some origins end up snapping to the same nearest node, therefore the code below groups and sums origin populations
origins_w_demands_series_no_dupl = origins_w_demands_series.groupby('NN').sum()
len(origins_w_demands_series_no_dupl)

678

### we produced a series that has each origin and its respective population

In [29]:
origins_w_demands_series_no_dupl

NN
3       1458.0
21      2232.0
32      2041.0
82      1508.0
84      1610.0
99      1295.0
106     1216.0
114      824.0
124      440.0
130     1104.0
149     1798.0
150     2484.0
152      310.0
161      665.0
163     1611.0
170     2729.0
173     2208.0
174     2426.0
175     1225.0
222      526.0
224      455.0
294      649.0
295     2393.0
296     2218.0
297     2301.0
298     2689.0
321      512.0
335     1303.0
351     2787.0
352     1978.0
         ...  
6440    1525.0
6478     297.0
6485    1510.0
6486    2716.0
6493    2990.0
6495    2664.0
6498     998.0
6513    1579.0
6514     852.0
6528    1078.0
6529    2113.0
6533     130.0
6536    2718.0
6546     261.0
6558    1151.0
6568    2352.0
6605    2816.0
6626    2503.0
6629    2505.0
6645    2854.0
6659    1183.0
6660     682.0
6666    1451.0
6671    1818.0
6674    2150.0
6675    1092.0
6698    2078.0
6703     435.0
6718    1285.0
6726    2981.0
Length: 678, dtype: float64

In [31]:
#for Julia functions
origins_w_demands_series_no_dupl.to_csv(os.path.join(pth, 'origins_w_demands_series_no_dupl.csv'), header=["pop"])

### Inputs include the OD matrix, locating 3 facilities, a pre-specified maximum distance coverage of 2000 seconds, and a series of origins with their population

In [24]:
import importlib
importlib.reload(gn)

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.9 


<module 'GOSTnet_Optimization' from '../../../GOSTNets/GOSTNets/GOSTnet_Optimization.py'>

In [25]:
max_coverage_result = gn.optimize_max_coverage(OD_df, p_facilities = 3, max_coverage = 2000, origins_pop_series = origins_w_demands_series_no_dupl, existing_facilities = None)

print objective value
1058482.0


In [26]:
max_coverage_result

[4647, 474, 6107]